In [ ]:
#| default_exp custom_dataset

In [ ]:
# | export
from typing import Tuple
from pathlib import Path
import os
import random

from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

from Multi_alphabet_Text_Recognition.xml_parser import read_kaist_xml
from Multi_alphabet_Text_Recognition.cropping import crop_text_from_image_korean

In [ ]:
# | export
class CustomDataset(Dataset):
    
    def __init__(self, targ_dir: str, dataset: str, transform=None) -> None:
        if dataset == 'arab':
            pass
#             self.annotations = generate_dataset_instances_eng_arab(targ_dir)
        elif dataset == 'korean':
            self.annotations = generate_dataset_instances_korean(targ_dir)
            
        self.transform = transform
        
        self.default_transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def load_image(self, index: int) -> Image.Image:
        image_path = self.annotations[index][0]
        return Image.open(image_path) 
    
    def __len__(self) -> int:
        return len(self.annotations)
    
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        img = self.load_image(index)
        transcript = self.annotations[index][1]

        if self.transform:
            return self.transform(img), transcript
        else:
            return self.default_transform(img), transcript

In [ ]:
# | export

def generate_dataset_instances_korean(path: Path) -> Tuple[str, str]:
    return_tuple_list = []
    index = 0
    for image_name in Path(path).glob('*.jpg'):
        xml_path = str(image_name)[0:-3] + 'xml'
        words = read_kaist_xml(xml_path)
        if words[1] == True:
            for i in range(len(words[0])):
                if words[0][i] == '':
                    continue
                image_save_path = '../../datasets/korean/KAIST_cropped/' + str(index) + '.jpg'
                crop_text_from_image_korean(str(image_name), words[2][i][0], words[2][i][1], words[2][i][2], words[2][i][3], image_save_path)
                index += 1
                return_tuple_list.append((image_save_path, words[0][i]))
            
    return return_tuple_list

def plot_transformed_images(image_paths, transform, n=3, seed=42):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)
            
        
def display_random_images(dataset: torch.utils.data.dataset.Dataset,
                          n: int = 2,
                          display_shape: bool = True,
                          seed: int = None):
    
    # 2. Adjust display if n too high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For display purposes, n shouldn't be larger than 10, setting to 10 and removing shape display.")
    
    # 3. Set random seed
    if seed:
        random.seed(seed)

    # 4. Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. Setup plot
    plt.figure(figsize=(16, 8))

    # 6. Loop through samples and display random samples 
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_transcription = dataset[targ_sample][0], dataset[targ_sample][1]

        # 7. Adjust image tensor shape for plotting: [color_channels, height, width] -> [color_channels, height, width]
        targ_image_adjust = targ_image.permute(1, 2, 0)

        # Plot adjusted samples
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        print(targ_transcription)
        if display_shape:
            title = f"\nshape: {targ_image_adjust.shape}"
            plt.title(title)
    plt.show()